In [2]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

In [6]:
model_id = "NousResearch/Meta-Llama-3.1-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto',
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [8]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [9]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [10]:
model.config

LlamaConfig {
  "_name_or_path": "NousResearch/Meta-Llama-3.1-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false

Now we need to choose a data set and download it so that we can use it, I will be using the mteb/legal-summarization dataset

In [11]:
!git clone https://github.com/lauramanor/legal_summarization

Cloning into 'legal_summarization'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 31 (delta 2), reused 0 (delta 0), pack-reused 25 (from 1)
Receiving objects: 100% (31/31), 136.60 KiB | 8.04 MiB/s, done.
Resolving deltas: 100% (10/10), done.


This will load the dataset as a JSON object

In [12]:
import json

jsonl_array = []

with open('legal_summarization/tldrlegal_v1.json') as f:
  data = json.load(f)
  for key, value in data.items():
    jsonl_array.append(value)

In [15]:
print(len(jsonl_array))

85


Now we can convert the JSON docs into a dataset using HuggingFace

In [16]:
from datasets import Dataset, load_dataset

legal_dataset = Dataset.from_list(jsonl_array)

In [17]:
print(legal_dataset)

Dataset({
    features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
    num_rows: 85
})


Now we split the dataset into a training test split.  


*   First we split the data set beetwen train and test (80% - 20%)
*   Second we split the "test" portion further to get a test set and a validation test
*   Last, we set the three datasets into a dataset dictionary



In [18]:
legal_dataset = legal_dataset.train_test_split(test_size=0.2)

In [19]:
legal_dataset_test_valid = legal_dataset["test"].train_test_split(test_size=0.5)

In [20]:
from datasets import DatasetDict

legal_dataset = DatasetDict({
    "train" : legal_dataset["train"],
    "test" : legal_dataset_test_valid["test"],
    "validation" : legal_dataset_test_valid["train"]
})

In [21]:
print(legal_dataset)

DatasetDict({
    train: Dataset({
        features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
        num_rows: 68
    })
    test: Dataset({
        features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
        num_rows: 9
    })
    validation: Dataset({
        features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
        num_rows: 8
    })
})


In [26]:
legal_dataset["validation"][7]

{'doc': 'Dropbox Terms of Service',
 'id': '531ca8704a169de51d00004b',
 'original_text': 'thanks for using dropbox. these terms of service terms cover your use and access to the services client software and websites services provided by dropbox inc our privacy policy explains how we collect and use your information while our acceptable use policy outlines your responsibilities when using our services. by using our services you re agreeing to be bound by these terms and to review our privacy and acceptable use policies. if you re using our services for an organization you re agreeing to these terms on behalf of that organization.',
 'reference_summary': 'by accepting these terms you also agree to the privacy and acceptable use policies linked in the right.',
 'title': 'Introduction',
 'uid': 'legalsum76'}

Now we need to format the instruction prompt for training purposes

In [27]:
INSTRUCTION_PROMPT_TEMPLATE = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please convert the following legal content into a short human-readable summary<|eot_id|><|start_header_id|>user<|end_header_id|>

[LEGAL_DOC]{LEGAL_TEXT}[END_LEGAL_DOC]<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

RESPONSE_TEMPLATE = """\
{NATURAL_LANGUAGE_SUMMARY}<|eot_id|><|end_of_text|>"""

In [28]:
def create_prompt(sample, include_response = True):
  """
  Parameters:
    - sample: dict representing row of dataset
    - include_response: bool

  Functionality:
    This function should build the Python str `full_prompt`.

    If `include_response` is true, it should include the summary -
    else it should not contain the summary (useful for prompting) and testing

  Returns:
    - full_prompt: str
  """
  full_prompt = INSTRUCTION_PROMPT_TEMPLATE.format(LEGAL_TEXT=sample["original_text"])

  if include_response:
    full_prompt += RESPONSE_TEMPLATE.format(NATURAL_LANGUAGE_SUMMARY=sample["reference_summary"])


  return full_prompt

In [29]:
print(create_prompt(legal_dataset["validation"][7]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please convert the following legal content into a short human-readable summary<|eot_id|><|start_header_id|>user<|end_header_id|>

[LEGAL_DOC]thanks for using dropbox. these terms of service terms cover your use and access to the services client software and websites services provided by dropbox inc our privacy policy explains how we collect and use your information while our acceptable use policy outlines your responsibilities when using our services. by using our services you re agreeing to be bound by these terms and to review our privacy and acceptable use policies. if you re using our services for an organization you re agreeing to these terms on behalf of that organization.[END_LEGAL_DOC]<|eot_id|><|start_header_id|>assistant<|end_header_id|>by accepting these terms you also agree to the privacy and acceptable use policies linked in the right.<|eot_id|><|end_of_text|>


In [35]:
def generate_response(prompt, model, tokenizer):
  """
  Parameters:
    - prompt: str representing formatted prompt
    - model: model object
    - tokenizer: tokenizer object

  Functionality:
    This will allow our model to generate a response to a prompt!

  Returns:
    - str response of the model
  """

  # convert str input into tokenized input
  encoded_input = tokenizer(prompt,  return_tensors="pt")

  # send the tokenized inputs to our GPU
  model_inputs = encoded_input.to('cuda')

  # generate response and set desired generation parameters
  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=256,
      do_sample=True,
      pad_token_id=tokenizer.eos_token_id
  )

  # decode output from tokenized output to str output
  decoded_output = tokenizer.batch_decode(generated_ids)

  # return only the generated response (not the prompt) as output
  return decoded_output[0].split("<|end_header_id|>")[-1]

In [36]:
generate_response(create_prompt(legal_dataset["test"][1], include_response=False),
                  model,
                  tokenizer)

'\n\nHere is a short human-readable summary of the legal content:\n\n**Disclaimer and Liability**\n\n* TLDR provides its website\'s information and services "as is" without warranties of any kind, including accuracy, fitness for purpose, or non-infringement.\n* TLDR is not responsible for errors, interruptions, or defects in the website or its services.\n* TLDR is not liable for any damages, including loss of revenue, emotional distress, or other losses, resulting from using the website or services.\n* TLDR is not a law firm and does not provide legal advice. Using the website or services does not create an attorney-client relationship.\n* You are responsible for consulting your own legal counsel before using the website or services.\n* TLDR has no liability for any damage to your computer, data, or business resulting from the content on the website.\n* You agree to defend and indemnify TLDR from any claims arising from your use of the website, services, or content.\n\n**Key Points:**\

In [37]:
# Ground Truth Summary
legal_dataset["test"][1]["reference_summary"]

'we re not a law firm nor liable for the content on the site and offer no warranty.'

In [38]:
generate_response(create_prompt(legal_dataset["test"][3], include_response=False),
                  model,
                  tokenizer)

"\n\nHere's a short, human-readable summary of the legal content:\n\n**Contribution Terms**\n\nTo contribute to the website, you must have a registered account. You retain the copyright to your contributions, but by sharing them, you grant the website a license to use them under a Creative Commons BY license. This means:\n\n* You promise to have the right to contribute the content and that it doesn't infringe on anyone's rights.\n* You give the website permission to use your contribution in any way they see fit.\n* You agree to attribute your contribution as part of the collective work, but the website doesn't have to specifically credit you.\n* You release the website from the obligation to attribute your contributions to you.\n* You're responsible for including copyright notices and licenses for your contributions.\n* The website may review and reject or delete any contribution that violates these terms or is deemed objectionable.\n* By contributing, you grant the website a royalty-f

In [39]:
# Ground Truth Summary
legal_dataset["test"][3]["reference_summary"]

'you must own the content that you contribute and release it under our cc by terms with tldrlegal as the attribution party. for simplicity when people link back you allow them to mention only tldrlegal explicitly. tldr is allowed to publish and adapt this content.'

Post-processing

In [40]:
from peft import prepare_model_for_kbit_training
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

In [41]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [42]:
from peft import LoraConfig, get_peft_model

# set our rank (higher value is more memory/better performance)
lora_r = 16

# set our dropout (default value)
lora_dropout = 0.1

# rule of thumb: alpha should be (lora_r * 2)
lora_alpha = 32

# construct our LoraConfig with the above hyperparameters
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM"
)

In [43]:
model = get_peft_model(
    model,
    peft_config
)

print_trainable_parameters(model)

trainable params: 41943040 || all params: 4582543360 || trainable%: 0.9152786281546499


In [44]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

In [45]:
from trl import SFTConfig

max_seq_length = 1024

args = SFTConfig(
  output_dir = "llama381binstruct_summarize_short",
  #num_train_epochs=5,
  max_steps = 400, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 1,
  warmup_steps = 30,
  logging_steps=10,
  #evaluation_strategy="epoch",
  eval_strategy="steps",
  eval_steps=25, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  lr_scheduler_type='linear',
  dataset_kwargs={
        "add_special_tokens" : False,
        "append_concat_token" : False,
  },
  max_seq_length=max_seq_length,
  packing=True,
)

In [46]:
from trl import SFTTrainer

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  tokenizer=tokenizer,
  formatting_func=create_prompt,
  args=args,
  train_dataset=legal_dataset["train"],
  eval_dataset=legal_dataset["validation"]
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:520: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:431: UserWarning: You passed `packing=True` to the SFTTrainer/SFTConfig, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [47]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
25,1.747600,1.485255
50,0.736700,1.563995
75,0.377400,1.747550
100,0.142900,1.999268
125,0.086000,2.053135
150,0.037500,2.194381
175,0.022400,2.323425
200,0.009600,2.174317
225,0.005300,2.267569
250,0.003500,2.401878


TrainOutput(global_step=400, training_loss=0.22596230145078153, metrics={'train_runtime': 2112.0024, 'train_samples_per_second': 0.189, 'train_steps_per_second': 0.189, 'total_flos': 1.85471820890112e+16, 'train_loss': 0.22596230145078153, 'epoch': 21.05263157894737})

In [48]:
from huggingface_hub import notebook_login

notebook_login()

In [52]:
trainer.push_to_hub("ai-maker-space/llama381binstruct_summarize_short")

events.out.tfevents.1726262774.b18f3a336bb8.972.0:   0%|          | 0.00/19.1k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Gerardofer/llama381binstruct_summarize_short/commit/0fea22fac28390fbbfd723b75a5b32cc557620f3', commit_message='ai-maker-space/llama381binstruct_summarize_short', commit_description='', oid='0fea22fac28390fbbfd723b75a5b32cc557620f3', pr_url=None, pr_revision=None, pr_num=None)

In [53]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [54]:
generate_response(create_prompt(legal_dataset["test"][1], include_response=False),
                  merged_model,
                  tokenizer)

'\n\nHere\'s a short human-readable summary of the legal content:\n\n**Disclaimer and Limitation of Liability**\n\nTLDR provides its website and services "as is" without warranties of any kind. This means that TLDR doesn\'t guarantee that the information, services, or websites will be:\n\n* Accurate or complete\n* Free from errors or viruses\n* Uninterrupted or error-free\n* Suitable for a particular purpose\n\n**Limitation of Liability**\n\nIn no event will TLDR or its affiliates be liable for:\n\n* Any damages, including indirect, consequential, or punitive damages\n* Losses or expenses resulting from using the website or services\n* Claims of copyright, patent, or trademark infringement\n* Conduct of third parties\n\n**No Attorney-Client Relationship**\n\nUsing TLDR\'s website or services does not create an attorney-client relationship. TLDR is not a law firm and does not provide legal advice. You should consult your own lawyer before using TLDR\'s services.\n\n**Indemnification**\n

In [57]:
legal_dataset["test"][3]["original_text"]

'the sites may offer you opportunities to contribute content contribution s. in order to make a contribution to the sites you must have a registered account. the owner of any contribution retains copyright rights to such content. if you make a contribution you must have the right to contribute this contribution under cc by either through ownership of the copyright because the contribution is in the public domain or through a license that grants you this right consistent with these terms. if your contribution is protected by copyright then unless it is licensed under cc by or under a license which grants you the right to contribute it under cc by you must not provide the contribution to tldr. you warrant represent and assert that you have the right to provide the contribution that no contribution violates or infringes the rights of any third party including copyright trademark privacy publicity or any other personal or proprietary rights that the contribution does not constitute confide

In [56]:
generate_response(create_prompt(legal_dataset["test"][3], include_response=False),
                  merged_model,
                  tokenizer
                  )

"\n\nHere is a short, human-readable summary of the legal content:\n\n**Contributing to the Sites**\n\nTo contribute content to the sites, you need to have a registered account. You retain the copyright to your contributions, but by providing them, you grant the sites a license under the Creative Commons BY license (CC BY). This license allows the sites to use, modify, and distribute your contributions without needing to ask for permission.\n\n**Your Responsibilities**\n\n* You must have the right to contribute your work and not infringe on anyone else's rights.\n* You warrant that your contribution is original and doesn't contain confidential or defamatory content.\n* You grant the sites a license to use your contribution under CC BY, even if you license it differently elsewhere.\n* You agree to include copyright notices and licenses for your contributions.\n* You are responsible for creating backup copies of your contributions.\n\n**Site's Rights**\n\n* The sites may review and rejec

In [58]:
legal_dataset["test"][3]["reference_summary"]

'you must own the content that you contribute and release it under our cc by terms with tldrlegal as the attribution party. for simplicity when people link back you allow them to mention only tldrlegal explicitly. tldr is allowed to publish and adapt this content.'